This notebook was used to extract features from the neural network for images

In [1]:
%matplotlib inline
#general imports
import os
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import pylab
import pandas as pd
from datetime import datetime, time, timedelta
import tables
import matplotlib
import matplotlib.pyplot as plt
from datetime import time, datetime, timedelta

#text imports
import gensim
from gensim.models import word2vec
import multiprocessing
cores = multiprocessing.cpu_count()
import random
assert gensim.models.word2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

#images imports
import tensorflow as tf
from scipy.misc import imread, imresize

/Users/christophelanternier/anaconda/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
import common.p_vgg16 as p_vgg16
import common.p_cca as p_cca
import common.p_w2v as p_w2v

In [3]:
#get all usefull paths

homePath = os.path.join(os.path.expanduser('~'), 'Documents', 'Cours', 'MVA', 'OBJR', 'FinalProject')
MODEL_PATH = os.path.join(homePath, 'model')+os.sep
IMAGES_PATH = os.path.join(homePath, 'images', 'train2014')+os.sep

## Initialize feature exctraction

Text

In [4]:
#Load pre-trained word2vec NN, takes a bit of time
model = gensim.models.Word2Vec.load_word2vec_format(MODEL_PATH+'GoogleNews-vectors-negative300.bin', binary=True)  

# initialize COCO api for caption annotations
dataType='train2014'
annFile = '%s/annotations/captions_%s.json'%(homePath,dataType)
coco_caps=COCO(annFile)

loading annotations into memory...
Done (t=2.15s)
creating index...
index created!


In [5]:
w2v = p_w2v.W2V(model, coco_caps)

Image

In [6]:
#initialize CNN
sess = tf.Session()
imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
vgg = p_vgg16.VGG16(imgs, MODEL_PATH + 'vgg16_weights.npz', sess)

0 conv1_1_W (3, 3, 3, 64)
1 conv1_1_b (64,)
2 conv1_2_W (3, 3, 64, 64)
3 conv1_2_b (64,)
4 conv2_1_W (3, 3, 64, 128)
5 conv2_1_b (128,)
6 conv2_2_W (3, 3, 128, 128)
7 conv2_2_b (128,)
8 conv3_1_W (3, 3, 128, 256)
9 conv3_1_b (256,)
10 conv3_2_W (3, 3, 256, 256)
11 conv3_2_b (256,)
12 conv3_3_W (3, 3, 256, 256)
13 conv3_3_b (256,)
14 conv4_1_W (3, 3, 256, 512)
15 conv4_1_b (512,)
16 conv4_2_W (3, 3, 512, 512)
17 conv4_2_b (512,)
18 conv4_3_W (3, 3, 512, 512)
19 conv4_3_b (512,)
20 conv5_1_W (3, 3, 512, 512)
21 conv5_1_b (512,)
22 conv5_2_W (3, 3, 512, 512)
23 conv5_2_b (512,)
24 conv5_3_W (3, 3, 512, 512)
25 conv5_3_b (512,)
26 fc6_W (25088, 4096)
27 fc6_b (4096,)
28 fc7_W (4096, 4096)
29 fc7_b (4096,)
30 fc8_W (4096, 1000)
31 fc8_b (1000,)


## Extract features

In [12]:
#Choose the number of samples you want in your data
nsamples = 82782
imgIds = coco_caps.getImgIds()

Save data

In [ ]:
#This part allows to save the data by batches of 10000 (allows not to lose everything in case of a buggf)
time1 = datetime.now()

text_data = np.zeros((10000, 300))
image_data = np.zeros((10000, 4096))
save = True

for i in range(nsamples+1):
    #Text Features
    j = i%10000
    img = coco_caps.loadImgs(imgIds[i])[0]
    text_data[j,:] = w2v.get_text_representations(imgIds[i])

    #image Features
    img1 = imread(IMAGES_PATH+img['file_name'], mode='RGB')
    img1 = imresize(img1, (224, 224))
    prob = sess.run(vgg.fc1, feed_dict={vgg.imgs: [img1]})[0]
    # To try other fc layers in the NN:
    #prob = sess.run(vgg.fc2, feed_dict={vgg.imgs: [img1]})[0]
    image_data[j, :] = prob

    if i%100 ==0:
        time2 = datetime.now()
        td = time2-time1
        print(td)
        print str(i)
    
    if (i+1)%10000 == 0 and save == True:
        hdf5_path = "data/data_fc1_part_"+str((i+1)//10000)+".hdf5"
        hdf5_file = tables.open_file(hdf5_path, mode='w')
        text_storage = hdf5_file.create_array(hdf5_file.root, 'text', text_data)
        image_storage = hdf5_file.create_array(hdf5_file.root, 'image', image_data)
        hdf5_file.close()
        print("PART "+str(i//10000)+" done")
        text_data = np.zeros((10000, 300))
        image_data = np.zeros((10000, 4096))
            
#np.savetxt('data/text_data_'+str(layers_string[k])+'.txt', text_data)
#np.savetxt('data/image_data_'+str(layers_string[k])+'.txt', image_data)

0:00:00.567808
0
0:00:55.444678
100


In [ ]:
image_data = image_data[:2783]
text_data = text_data[:2783]

hdf5_path = "data/data_fc1_part_9.hdf5"
hdf5_file = tables.open_file(hdf5_path, mode='w')
text_storage = hdf5_file.create_array(hdf5_file.root, 'text', text_data)
image_storage = hdf5_file.create_array(hdf5_file.root, 'image', image_data)
hdf5_file.close()

Rejoin various batches of the data

In [ ]:
text_data = np.zeros((82783, 300))
image_data = np.zeros((82783, 4096))
for i in range(1,9):
    #hdf5_path = "data/data_fc2_part_1.hdf5"
    hdf5_path = "data/data_fc1_part_"+str(i)+".hdf5"
    read_hdf5_file = tables.open_file(hdf5_path, mode='r')
    # Here we slice [:] all the data back into memory, then operate on it
    text_data[(i-1)*10000:i*10000, :] = read_hdf5_file.root.text[:]
    image_data[(i-1)*10000:i*10000, :] = read_hdf5_file.root.image[:]

    read_hdf5_file.close()

hdf5_path = "data/data_fc1_part_9.hdf5"
read_hdf5_file = tables.open_file(hdf5_path, mode='r')
# Here we slice [:] all the data back into memory, then operate on it
text_data[80000:82783, :] = read_hdf5_file.root.text[:]
image_data[80000:82783, :] = read_hdf5_file.root.image[:]

read_hdf5_file.close()

Save final file

In [ ]:
hdf5_path = "data/data_fc1_all.hdf5"
hdf5_file = tables.open_file(hdf5_path, mode='w')
text_storage = hdf5_file.create_array(hdf5_file.root, 'text', text_data)
image_storage = hdf5_file.create_array(hdf5_file.root, 'image', image_data)
hdf5_file.close()

## Compute Category features for 3 view

In [ ]:
list_of_cats = []
cat_data = np.zeros((82783, 300))
image_without_cat = []
for i in range(82783):
    img = coco_caps.loadImgs(imgIds[i])[0]
    try:
        cat = dic_img_cat[img['id']]
        if len(cat.split(' ')) > 1:
            cat_array = []
            for word in cat.split(' '):
                cat_array.append(model[word])
            cat_data[i, :] = sum(cat_array)/len(cat_array)
        else:   
            cat_data[i, :] = model[cat]
    except:
        image_without_cat.append(i)    

## Compute Glove features

In [7]:
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

In [8]:
model = loadGloveModel('data/glove.6B/glove.6B.300d.txt')

Loading Glove Model
Done. 400000  words loaded!


In [9]:
w2v = p_w2v.W2V(model, coco_caps)

In [13]:
time1 = datetime.now()

text_data = np.zeros((nsamples+1, 300))

for i in range(nsamples+1):
    #Text Features
    text_data[i,:] = w2v.get_text_representations(imgIds[i])


In [16]:
hdf5_path = "data/data_glove.hdf5"
hdf5_file = tables.open_file(hdf5_path, mode='w')
text_storage = hdf5_file.create_array(hdf5_file.root, 'text', text_data)
hdf5_file.close()

In [23]:
imgId = imgIds[0]

In [26]:
annIds = coco_caps.getAnnIds(imgIds=imgId);
anns = coco_caps.loadAnns(annIds)
# Compute annotations mean (text word representation for one image)
global_mean = False
caption_vectors_mean = np.zeros(300)

caption_vectors = []
print anns[0]['caption'][:-1].split(' ')

[u'People', u'shopping', u'in', u'an', u'open', u'market', u'for', u'vegetables']


In [44]:
caption_vectors = []
for word in anns[0]['caption'][:-1].split(' '):
    caption_vectors.append(np.array(model[word.lower()]))
    caption_vectors_mean += sum(caption_vectors)/len(caption_vectors)